<a href="https://colab.research.google.com/github/nurulc/ATM/blob/master/tf_tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from google.colab import drive
import tensorflow_datasets as tfds
import os

# New Section

In [7]:

# drive.mount('content/gdrive')
head {housing-ma.csv}

SyntaxError: ignored